# Initial instance configuration

This guide walks you through the initial steps to get started with a fresh installation of a KAWA server on premise.

It requires an API key for an ADMIN user.


In [ ]:
from kywy.client.kawa_client import KawaClient as K

kawa = K.load_client_from_environment()
cmd = kawa.commands

## 1. Install the license

This allows to load a license file into a running kawa instance. License files are json files. If you need a license, please visit www.kawa.ai.



In [ ]:
kawa.commands.set_license(path_to_license_file='/tmp/path/to/license.json')

## 2. Change the setup admin password

In [ ]:
cmd.change_user_password('setup-admin@kawa.io', '********');

## 3. Activate the desired features

This will toggle the feature flags of the KAWA instance. The available flags are:

- data-patching (Allows users to edit values within the grid views)
- list-principals (Allow to show all the users in user selection lists - like members of workspaces etc..)
- scripts (Enable script support)
- automations (Enable automation support)
- dashboard-export (Enable dashboard export)
- data-sample (Shows a list of samples in the datasource section)

In [ ]:
cmd.toggle_feature(feature_name='data-samples', is_enabled=True)
cmd.toggle_feature(feature_name='automations', is_enabled=True);
cmd.toggle_feature(feature_name='scripts', is_enabled=True);

## 4. Configure the SMTP server (optional)

This step is required only if you want KAWA to be able to send emails through your company SMTP server.
The credentials of the SMTP server are configured during the isntallation process.

__Please note that the server needs to be restarted for the SMTP parameters to be taken in account.__

In [ ]:
# Turn on SMTP communications
# To turn back off, use: LOG instead of SMTP
cmd.replace_communication_provider_type('SMTP')

# Configure the SMTP server
# Use extraSmtpProperties to put any additional configuration from
#  https://javadoc.io/doc/com.sun.mail/jakarta.mail/latest/jakarta.mail/com/sun/mail/smtp/package-summary.html
cmd.replace_configuration('SmtpConfig', {
    'host': 'smtp.wayne.com',
    'port': 2525,
    'sslEnabled': False,
    'startTlsEnabled': True,
    'authenticationEnabled': True,
    'extraSmtpProperties': {
        "mail.smtp.ssl.trust": "some-domain.com"
    }
})

# Defines a subject + sender for email activation emails
cmd.replace_configuration('EmailActivatorConfiguration', {
    'sender': 'postmaster@kawa-analytics.com',
    'subject': 'One last step to activate your account'
})

# Defines a subject + sender for autimation emails
cmd.replace_configuration('AutomationBehaviourConfiguration', {
    'emailSender': 'robot@kawa-analytics.com',
    'emailSubject': 'New automation from KAWA'
});

## 5. Connect a Python script runner (optional)

Repeat this step for each workspace you wish to connect a python runner to.
It will allow users from this workspace to benefit from Python columns and Python automations.

In [ ]:
# Will activate a runner in this workspace
WORKSPACE_ID=1
kawa.set_active_workspace_id(workspace_id=WORKSPACE_ID)


cmd.add_script_runner(
    name='Main runner',
    host='kawa-script-runner',
    port=8815,
    tls=False
)

print(kawa.runner_health())

## 6. Configure SSO (optional)

KAWA supports SAML based SSO authentication. Use these step to configure it.
By default, KAWA will use its own authentication mechanism and act as an IDP.

__Please note that the server needs to be restarted for the Authentication parameters to be taken in account.__

In [ ]:
cmd.replace_configuration('SamlConfiguration', {
    'nameClaim': 'http://schemas.xmlsoap.org/ws/2005/05/identity/claims/name',
    'emailClaim': 'http://schemas.xmlsoap.org/ws/2005/05/identity/claims/emailaddress',
    'idpMetadataUrl': 'https://portal.sso.us-east-1.amazonaws.com/saml/metadata/abcdef1234567890',
    'serviceProviderHost': 'https://kawa.wayne.io',
})

cmd.replace_configuration('GlobalAuthenticationConfiguration', {
  'setupAdminUniqueId': 'setup-admin@kawa.io',
  'authenticationMethod': 'SAML'
});